In [ ]:
!pip install scikeras[tensorflow]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import itertools
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import make_scorer
import concurrent.futures
from threading import Lock

from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
import keras.backend as K
import tensorflow as tf
from scikeras.wrappers import KerasRegressor

In [ ]:
!pip freeze > requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Preprocessing

Load the CUP dataset

In [ ]:
header=['ID','a','b','c','d','e','f', 'g', 'h', 'i', 'j','Class_x', 'Class_y']
df = pd.read_csv("/content/drive/Shareddrives/ML/CUP/ML-CUP21-TR.csv", header=None,delimiter=',', skiprows=7,names=header)
df.index=df['ID'].values
df.drop('ID', axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
col=[c for c in df.columns if (c!='Class_x' and c!= 'Class_y')]
x= df[col].values
y= df[['Class_x', 'Class_y']].values

Training/Test splitting with Hold-out approch (90%-10%)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)
print(y_train.shape, y_test.shape)

MEE definition

In [ ]:
def mean_euclidean_error_tf(y_true, y_pred):
    return K.mean(K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1)))

In [ ]:
def mean_euclidean_error(y_true, y_pred):
    return np.mean(np.sqrt(np.sum(np.square(y_pred-y_true), axis=-1)))

In [ ]:
score = make_scorer(mean_euclidean_error, greater_is_better = False)

In [ ]:
def build_model(weight_init=0.2, weight_distr=0, activ='relu',layer=1, unit=4, eta=0.2, alpha=0.5, lambd=0):
  
  tf.random.set_seed(0)  
    
  if weight_distr==0:
    init= tf.keras.initializers.RandomUniform(minval=-weight_init, maxval=weight_init)
  elif weight_distr==1:
    init= tf.keras.initializers.RandomNormal(mean=0., stddev=weight_init)
  else:
    init= tf.keras.initializers.GlorotNormal()

  reg= tf.keras.regularizers.l2(l2=lambd)


  model= tf.keras.models.Sequential()
  model.add(tf.keras.layers.Input(10,))
  for i in range(layer):
    model.add(tf.keras.layers.Dense(unit, activation='tanh', kernel_initializer=init, bias_initializer=init, kernel_regularizer=reg))
  model.add(tf.keras.layers.Dense(2, activation='linear', kernel_initializer=init, bias_initializer=init, kernel_regularizer=reg))

  loss=mean_euclidean_error_tf
  opt= tf.keras.optimizers.SGD(learning_rate=eta, momentum=alpha, nesterov=False)
  metric=mean_euclidean_error_tf
  model.compile(loss=loss, 
                optimizer=opt,
                metrics=[metric])
  
  #print(model.get_weights())
  return model

In [ ]:
def build_model_Adam(weight_init=0.2, weight_distr=0, activ='relu',layer=1, unit=4, eta=0.2, alpha=0.5, lambd=0, beta_1=0.9, beta_2=0.999, epsilon=0.0000001):
  
  tf.random.set_seed(0)  
    
  if weight_distr==0:
    init= tf.keras.initializers.RandomUniform(minval=-weight_init, maxval=weight_init)
  elif weight_distr==1:
    init= tf.keras.initializers.RandomNormal(mean=0., stddev=weight_init)
  else:
    init= tf.keras.initializers.GlorotNormal()

  reg= tf.keras.regularizers.l2(l2=lambd)


  model= tf.keras.models.Sequential()
  model.add(tf.keras.layers.Input(10,))
  for i in range(layer):
    model.add(tf.keras.layers.Dense(unit, activation='tanh', kernel_initializer=init, bias_initializer=init, kernel_regularizer=reg))
  model.add(tf.keras.layers.Dense(2, activation='linear', kernel_initializer=init, bias_initializer=init, kernel_regularizer=reg))

  loss=mean_euclidean_error_tf
  opt= tf.keras.optimizers.Adam(learning_rate=eta, beta_1=beta_1, beta_2=beta_2,epsilon=epsilon)
  metric=mean_euclidean_error_tf
  model.compile(loss=loss, 
                optimizer=opt,
                metrics=[metric])
  
  #print(model.get_weights())
  return model

In [ ]:
class haltCallback(tf.keras.callbacks.Callback):
    def __init__(self, err):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.err=err

    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('mean_euclidean_error_tf') <=self.err):
            self.model.stop_training = True

## Stacking

Ensemble of base estimators with final Ridge regressor on top of their predictions

In [ ]:
class Stacking():

  #Stacking constructor which takes the best grid's output hyper-parameters for
  #the estimators to ensamble
  def __init__(self) -> None:
      nn_param_batch={
          'weight_init': 0.4,
          'weight_distr': 1,
          'unit': 40,
          'layer': 3,
          'eta': 0.03,
          'alpha': 0.95,
          'lambd': 0.001,
          'activ': 'tanh',
      }
      mean_err_batch= 0.8342830985784531  #tr
      trainingStopCallback_batch = haltCallback(mean_err_batch)

      nn_param_mb={
          'weight_init': 0.2,
          'weight_distr': 1,
          'unit': 50,
          'layer': 3,
          'eta': 0.025,
          'alpha': 0,
          'lambd': 0.0005,
          'activ': 'tanh'
      }
      mean_err_mb= 1.0076722502708435
      trainingStopCallback_mb = haltCallback(mean_err_mb)


      nn_param_adam={
          'weight_init': 0.4,
          'weight_distr': 1,
          'unit': 40,
          'layer': 3,
          'eta': 0.025,
          'beta_1': 0.9,
          'beta_2': 0.7,
          'lambd': 0.001,
          'activ': 'tanh'
      }
      mean_err_adam= 0.8765210807323456
      trainingStopCallback_adam = haltCallback(mean_err_adam)


      nn_param_online={
          'weight_init': 0.2,
          'weight_distr': 1,
          'unit': 50,
          'eta': 0.0005,
          'alpha': 0,
          'layer': 2,
          'lambd': 0.001,
          'activ': 'tanh'
      }
      mean_err_online= 0.9664344638586044
      trainingStopCallback_online = haltCallback(mean_err_online)

      #estimator definition with best hyper-parameters
      self.estimators=[
            ('knn', KNeighborsRegressor(n_neighbors= 9, p=1, weights='distance')),
            #('svm', MultiOutputRegressor(SVR(C= 10, epsilon= 0.5, gamma= 'auto', kernel= 'rbf'))),
            #('lbe', Pipeline(steps=[('lbe',PolynomialFeatures(degree=4, interaction_only=True)),('ridge',Ridge(alpha=10, solver='saga', random_state=0))])),
            ('batch', KerasRegressor(build_model, **nn_param_batch, epochs=800, shuffle=True, batch_size=len(x_train), verbose=0, callbacks=[trainingStopCallback_batch])),
            #('mb', KerasRegressor(build_model, **nn_param_mb, epochs=800, shuffle=True, batch_size=100, verbose=0, callbacks=[trainingStopCallback_mb])),
            ('adam', KerasRegressor(build_model_Adam, **nn_param_adam, epochs=800, shuffle=True, batch_size=len(x_train), verbose=0, callbacks=[trainingStopCallback_adam])),
            #('online', KerasRegressor(build_model, **nn_param_online, epochs=400, shuffle=True, batch_size=1, verbose=0, callbacks=[trainingStopCallback_online])),
            ('forest', RandomForestRegressor(max_depth=20, random_state=0, n_estimators=100, criterion='squared_error', min_samples_split=2, min_samples_leaf=2, 
                                    max_features = 3, bootstrap=False))
      ]

      #Ridge as final estimator on top of the base estimators' outputs
      #the RidgeCV class has a built-in grid-search in order to tune the alpha parameter
      self.final_estimator_x= RidgeCV(alphas=(0.001,0.01,0.1,1,10,25,50,75,100,200), scoring='neg_mean_squared_error', cv=5)
      self.final_estimator_y= RidgeCV(alphas=(0.001,0.01,0.1,1,10,50,75,100,150,200,250,300), scoring='neg_mean_squared_error', cv=5)
      
      self.col= [name for (name, estimator) in self.estimators]

  #In the fit method we first train each base estimator with a cross validated approch.
  #Then we build a DataFrame made by base estimator predictions on which we tune our Ridge models (one for each target x/y)
  def fit(self, X_train, y_train):
      pred_train_x= pd.DataFrame(columns=self.col)
      pred_train_y= pd.DataFrame(columns=self.col)

      for (name, estimator) in self.estimators:
        pred= cross_val_predict(estimator, X_train, y_train, cv=4)
        pred_train_x[name]= pred[:,0]
        pred_train_y[name]= pred[:,1]

      val_mee=[]
      kf = KFold(n_splits=4, shuffle=True, random_state=0)
      #############!!!!!!!!!!!###########
      for train, val in kf.split(pred_train_x):
        self.final_estimator_x.fit(pred_train_x.loc[train], y_train[train,0]) #chiamaRidgeCV
        pred_val_x= self.final_estimator_x.predict(pred_train_x.loc[val])

        self.final_estimator_y.fit(pred_train_y.loc[train], y_train[train,1])
        pred_val_y= self.final_estimator_y.predict(pred_train_y.loc[val])

        print(self.final_estimator_x.alpha_, self.final_estimator_y.alpha_)
        print(mean_euclidean_error(y_train[val], np.column_stack((pred_val_x, pred_val_y))))
        val_mee.append(mean_euclidean_error(y_train[val], np.column_stack((pred_val_x, pred_val_y))))
        
      self.final_estimator_x.fit(pred_train_x, y_train[:,0])
      self.final_estimator_y.fit(pred_train_y, y_train[:,1])

      return (np.mean(val_mee), np.std(val_mee), self.final_estimator_x.alpha_, self.final_estimator_y.alpha_)

  #In the predict method we first retrain the base estimators on the whole TR
  #And then we use the trained models to predict the TS, building the DataFrame on which Ridge predicts the final outputs
  def predict(self, X_train, y_train, X_test):
      pred_test_x= pd.DataFrame(columns=self.col)
      pred_test_y= pd.DataFrame(columns=self.col)

      for (name, estimator) in self.estimators:
        estimator.fit(X_train, y_train)
        pred_test= estimator.predict(X_test)
        pred_test_x[name]= pred_test[:,0]
        pred_test_y[name]= pred_test[:,1]

      return np.column_stack((self.final_estimator_x.predict(pred_test_x), self.final_estimator_y.predict(pred_test_y)))

In [ ]:
class Stacking():

  #Stacking constructor which takes the best grid's output hyper-parameters for
  #the estimators to ensamble
  def __init__(self) -> None:
      nn_param_batch={
          'weight_init': 0.4,
          'weight_distr': 1,
          'unit': 40,
          'layer': 3,
          'eta': 0.03,
          'alpha': 0.95,
          'lambd': 0.001,
          'activ': 'tanh',
      }
      mean_err_batch= 0.8342830985784531
      trainingStopCallback_batch = haltCallback(mean_err_batch)

      nn_param_mb={
          'weight_init': 0.2,
          'weight_distr': 1,
          'unit': 50,
          'layer': 3,
          'eta': 0.025,
          'alpha': 0,
          'lambd': 0.0005,
          'activ': 'tanh'
      }
      mean_err_mb= 1.0076722502708435
      trainingStopCallback_mb = haltCallback(mean_err_mb)


      nn_param_adam={
          'weight_init': 0.4,
          'weight_distr': 1,
          'unit': 40,
          'layer': 3,
          'eta': 0.025,
          'beta_1': 0.9,
          'beta_2': 0.7,
          'lambd': 0.001,
          'activ': 'tanh'
      }
      mean_err_adam= 0.8765210807323456
      trainingStopCallback_adam = haltCallback(mean_err_adam)


      nn_param_online={
          'weight_init': 0.2,
          'weight_distr': 1,
          'unit': 50,
          'eta': 0.0005,
          'alpha': 0,
          'layer': 2,
          'lambd': 0.001,
          'activ': 'tanh'
      }
      mean_err_online= 0.9664344638586044
      trainingStopCallback_online = haltCallback(mean_err_online)

      #estimator definition with best hyper-parameters
      self.estimators=[
            ('knn', KNeighborsRegressor(n_neighbors= 9, p=1, weights='distance')),
            #('svm', MultiOutputRegressor(SVR(C= 10, epsilon= 0.5, gamma= 'auto', kernel= 'rbf'))),
            #('lbe', Pipeline(steps=[('lbe',PolynomialFeatures(degree=4, interaction_only=True)),('ridge',Ridge(alpha=10, solver='saga', random_state=0))])),
            ('batch', KerasRegressor(build_model, **nn_param_batch, epochs=800, shuffle=True, batch_size=len(x_train), verbose=0, callbacks=[trainingStopCallback_batch])),
            #('mb', KerasRegressor(build_model, **nn_param_mb, epochs=800, shuffle=True, batch_size=100, verbose=0, callbacks=[trainingStopCallback_mb])),
            ('adam', KerasRegressor(build_model_Adam, **nn_param_adam, epochs=800, shuffle=True, batch_size=len(x_train), verbose=0, callbacks=[trainingStopCallback_adam])),
            #('online', KerasRegressor(build_model, **nn_param_online, epochs=400, shuffle=True, batch_size=1, verbose=0, callbacks=[trainingStopCallback_online])),
            ('forest', RandomForestRegressor(max_depth=20, random_state=0, n_estimators=100, criterion='squared_error', min_samples_split=2, min_samples_leaf=2, 
                                    max_features = 3, bootstrap=False))
      ]

      #Ridge as final estimator on top of the base estimators' outputs
      #the RidgeCV class has a built-in grid-search in order to tune the alpha parameter
      self.final_estimator_x= RidgeCV(alphas=(0.001,0.01,0.1,1,10,25,50,75,100,200), scoring='neg_mean_squared_error', cv=5)
      self.final_estimator_y= RidgeCV(alphas=(0.001,0.01,0.1,1,10,50,75,100,150,200,250,300), scoring='neg_mean_squared_error', cv=5)
      
      self.col= [name for (name, estimator) in self.estimators]

  #In the fit method we first train each base estimator with a cross validated approch.
  #Then we build a DataFrame made by base estimator predictions on which we tune our Ridge models (one for each target x/y)
  def fit(self, X_train, y_train):
      pred_train_x= pd.DataFrame(columns=self.col)
      pred_train_y= pd.DataFrame(columns=self.col)

      for (name, estimator) in self.estimators:
        pred= cross_val_predict(estimator, X_train, y_train, cv=4)
        pred_train_x[name]= pred[:,0]
        pred_train_y[name]= pred[:,1]

      self.final_estimator_x.fit(pred_train_x, y_train[:,0])  #find best alpha and retrain through RidgeCV
      self.final_estimator_y.fit(pred_train_y, y_train[:,1])

      val_mee=[]  
      ridge_x= Ridge(alpha=self.final_estimator_x.alpha_)   #imposto alpha
      ridge_y= Ridge(alpha=self.final_estimator_y.alpha_)
      kf = KFold(n_splits=4, shuffle=True, random_state=0)
      for train, val in kf.split(pred_train_x):
        ridge_x.fit(pred_train_x.loc[train], y_train[train,0])
        pred_val_x= ridge_x.predict(pred_train_x.loc[val])

        ridge_y.fit(pred_train_y.loc[train], y_train[train,1])
        pred_val_y= ridge_y.predict(pred_train_y.loc[val])

        print(mean_euclidean_error(y_train[val], np.column_stack((pred_val_x, pred_val_y))))
        val_mee.append(mean_euclidean_error(y_train[val], np.column_stack((pred_val_x, pred_val_y))))
        
      

      return (np.mean(val_mee), np.std(val_mee), self.final_estimator_x.alpha_, self.final_estimator_y.alpha_)

  #In the predict method we first retrain the base estimators on the whole TR
  #And then we use the trained models to predict the TS, building the DataFrame on which Ridge predicts the final outputs
  def predict(self, X_train, y_train, X_test):
      pred_test_x= pd.DataFrame(columns=self.col)
      pred_test_y= pd.DataFrame(columns=self.col)

      for (name, estimator) in self.estimators: #modifico test
        estimator.fit(X_train, y_train)
        pred_test= estimator.predict(X_test)
        pred_test_x[name]= pred_test[:,0]
        pred_test_y[name]= pred_test[:,1]

      return np.column_stack((self.final_estimator_x.predict(pred_test_x), self.final_estimator_y.predict(pred_test_y)))

In [ ]:
stk= Stacking()
(mean, std, alpha1, alpha2)= stk.fit(x_train, y_train)

In [ ]:
stk= Stacking()
(mean, std, alpha1, alpha2)= stk.fit(x_train, y_train)

In [ ]:
print(mean, std, alpha1, alpha2)

In [ ]:
print(mean, std, alpha1, alpha2)

In [ ]:
y_pred_train= stk.predict(x_train, y_train, x_train)

In [ ]:
mean_euclidean_error(y_train, y_pred_train)

In [ ]:
y_pred_test= stk.predict(x_train, y_train, x_test)

In [ ]:
mean_euclidean_error(y_test, y_pred_test)

#Voting

A Voting regressor is an ensemble which averages the individual predictions from base estimators to form a final prediction.

In [ ]:
class Voting():

  #Voting constructor which takes the best grid's output hyper-parameters for
  #the estimators to ensamble
  def __init__(self) -> None:
      nn_param_batch={
          'weight_init': 0.4,
          'weight_distr': 1,
          'unit': 40,
          'layer': 3,
          'eta': 0.03,
          'alpha': 0.95,
          'lambd': 0.001,
          'activ': 'tanh',
      }
      mean_err_batch= 0.8342830985784531
      trainingStopCallback_batch = haltCallback(mean_err_batch)

      nn_param_mb={
          'weight_init': 0.2,
          'weight_distr': 1,
          'unit': 50,
          'layer': 3,
          'eta': 0.025,
          'alpha': 0,
          'lambd': 0.0005,
          'activ': 'tanh'
      }
      mean_err_mb= 1.0076722502708435
      trainingStopCallback_mb = haltCallback(mean_err_mb)


      nn_param_adam={
          'weight_init': 0.4,
          'weight_distr': 1,
          'unit': 40,
          'layer': 3,
          'eta': 0.025,
          'beta_1': 0.9,
          'beta_2': 0.7,
          'lambd': 0.001,
          'activ': 'tanh'
      }
      mean_err_adam= 0.8765210807323456
      trainingStopCallback_adam = haltCallback(mean_err_adam)


      nn_param_online={
          'weight_init': 0.2,
          'weight_distr': 1,
          'unit': 50,
          'eta': 0.0005,
          'alpha': 0,
          'layer': 2,
          'lambd': 0.001,
          'activ': 'tanh'
      }
      mean_err_online= 0.9664344638586044
      trainingStopCallback_online = haltCallback(mean_err_online)

      #estimator definition with best hyper-parameters
      self.estimators=[
            ('knn', KNeighborsRegressor(n_neighbors= 9, p=1, weights='distance')),
            #('svm', MultiOutputRegressor(SVR(C= 10, epsilon= 0.5, gamma= 'auto', kernel= 'rbf'))),
            #('lbe', Pipeline(steps=[('lbe',PolynomialFeatures(degree=4, interaction_only=True)),('ridge',Ridge(alpha=10, solver='saga', random_state=0))])),
            ('batch', KerasRegressor(build_model, **nn_param_batch, epochs=800, shuffle=True, batch_size=len(x_train), verbose=0, callbacks=[trainingStopCallback_batch])),
            #('mb', KerasRegressor(build_model, **nn_param_mb, epochs=800, shuffle=True, batch_size=100, verbose=0, callbacks=[trainingStopCallback_mb])),
            ('adam', KerasRegressor(build_model_Adam, **nn_param_adam, epochs=800, shuffle=True, batch_size=len(x_train), verbose=0, callbacks=[trainingStopCallback_adam])),
            #('online', KerasRegressor(build_model, **nn_param_online, epochs=400, shuffle=True, batch_size=1, verbose=0, callbacks=[trainingStopCallback_online])),
            ('forest', RandomForestRegressor(max_depth=20, random_state=0, n_estimators=100, criterion='squared_error', min_samples_split=2, min_samples_leaf=2, 
                                    max_features = 3, bootstrap=False))
      ]
      
      self.col= [name for (name, estimator) in self.estimators]

  #In the fit method we first train each base estimator with a cross validated approch.
  #Then we build a DataFrame made by base estimator predictions on which we predict using the mean voting (one for each target x/y)
  def fit(self, X_train, y_train):
      pred_train_x= pd.DataFrame(columns=self.col, index=np.arange(0,len(X_train)))
      pred_train_y= pd.DataFrame(columns=self.col, index=np.arange(0,len(X_train)))

      val_mee=[]
      kf = KFold(n_splits=4, shuffle=True, random_state=0)
      for train, val in kf.split(X_train):
        for (name, estimator) in self.estimators:
          estimator.fit(X_train[train], y_train[train])
          pred_val= estimator.predict(X_train[val])

          pred_train_x.loc[val, name]= pred_val[:,0]
          pred_train_y.loc[val, name]= pred_val[:,1]
        
        val_mee.append(mean_euclidean_error(y_train[val], np.column_stack((np.mean(pred_train_x.loc[val], axis=1), np.mean(pred_train_y.loc[val], axis=1)))))


      return (np.mean(val_mee), np.std(val_mee))

  #In the predict method we first retrain the base estimators on the whole TR
  #And then we use the trained models to predict the TS, building the DataFrame on which we take the mean
  def predict(self, X_train, y_train, X_test):
      pred_test_x= pd.DataFrame(columns=self.col)
      pred_test_y= pd.DataFrame(columns=self.col)

      for (name, estimator) in self.estimators:
        estimator.fit(X_train, y_train)
        pred_test= estimator.predict(X_test)
        pred_test_x[name]= pred_test[:,0]
        pred_test_y[name]= pred_test[:,1]

      return np.column_stack((np.mean(pred_test_x.values, axis=1), np.mean(pred_test_y.values, axis=1)))

In [ ]:
stk= Voting()
(mean,std)= stk.fit(x_train, y_train)

In [ ]:
print(mean, std)

In [ ]:
y_pred_train= stk.predict(x_train, y_train, x_train)

In [ ]:
mean_euclidean_error(y_train, y_pred_train)

In [ ]:
y_pred_test= stk.predict(x_train, y_train, x_test)

In [ ]:
mean_euclidean_error(y_test, y_pred_test)